In [2]:
import numpy as np
import scipy
import sympy as sp

import plotly.graph_objects as go
from qiskit.opflow import I, X, Y, Z, H

from qaoa import *

import pickle
import time
import warnings
warnings.filterwarnings("ignore")

/var/folders/1z/z41sxch92wz_jmlsyy10q6680000gn/T/ipykernel_19541/3096987933.py:6: DeprecationWarning: The ``qiskit.opflow`` module is deprecated as of qiskit-terra 0.24.0. It will be removed no earlier than 3 months after the release date. For code migration guidelines, visit https://qisk.it/opflow_migration.
  from qiskit.opflow import I, X, Y, Z, H


In [3]:
def graph_energy(energy, points = None):
    res = len(energy)
    s = np.linspace(0, 2 * np.pi, res + 1)[:-1]
    
    fig = go.Figure(data=[
            go.Surface(x=s, y=s, z=energy, showscale=False,
                       colorscale='Viridis')])

    
    if points is not None:
        px, py, p_energy = points
        fig.add_trace(go.Scatter3d(x=px, y=py, z=p_energy,
                                   mode='markers'))

    fig.update_scenes(
        xaxis_title='Gamma',
        yaxis_title='Beta',
        zaxis_title='Energy'
    )

    fig.update_layout(title='p = 1 Energy Landscape',
                    autosize=False,
                    width=1000, height=700,
                    )
    fig.show()

In [4]:
"""
Schwinger
"""

select = {'N': 4, #qubits
          'g' : 1,  #coupling
          'm' : 1,  #bare mass
          'a' : 1, #lattice spacing
          'theta' : 0, #topological term
          'mixer_type' : 'Y', # type of mixer {'X', 'Y', 'XY'}
         }

model = QuantumModel('schwinger', 'Standard')


ham = model.make(select)
schwinger_target = ham.target
ham

driver:
4.0 * IIII
+ 2.0 * IIZZ
+ 1.0 * IZIZ
+ 1.0 * IZZI
+ 4.0 * IIIZ
- 1.0 * IIZI
+ 3.0 * IZII
- 2.0 * ZIII
mixer:
1.0 * IIIY
+ 1.0 * IIYI
+ 1.0 * IYII
+ 1.0 * YIII
target:
1.0 * IIII
+ 0.5 * IIZZ
+ 0.25 * IZIZ
+ 0.25 * IZZI
+ 1.0 * IIIZ
- 0.25 * IIZI
+ 0.75 * IZII
- 0.5 * ZIII
+ 0.25 * IIXX
+ 0.25 * IIYY
+ 0.25 * IXXI
+ 0.25 * IYYI
+ 0.25 * XXII
+ 0.25 * YYII

In [5]:
numqaoa = QAOA_Numpy(ham)

In [10]:
energy_scape = numqaoa.energy_landscape(10, 2)

/Users/riley/anaconda3/envs/qc/lib/python3.10/site-packages/scipy/sparse/linalg/_dsolve/linsolve.py:394: SparseEfficiencyWarning: splu converted its input to CSC format
  warn('splu converted its input to CSC format', SparseEfficiencyWarning)
/Users/riley/anaconda3/envs/qc/lib/python3.10/site-packages/scipy/sparse/linalg/_dsolve/linsolve.py:285: SparseEfficiencyWarning: spsolve is more efficient when sparse b is in the CSC matrix format
  warn('spsolve is more efficient when sparse b '
/Users/riley/anaconda3/envs/qc/lib/python3.10/site-packages/scipy/sparse/_index.py:103: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_intXint(row, col, x.flat[0])
/Users/riley/anaconda3/envs/qc/lib/python3.10/site-packages/scipy/sparse/linalg/_dsolve/linsolve.py:394: SparseEfficiencyWarning: splu converted its input to CSC format
  warn('splu converted its input to CSC format', SparseEfficiencyWarning)
/Users/riley/anacond

In [11]:
graph_energy(energy_scape[0,0], points = None)

In [22]:
numqaoa.energy(0.62831853, 0.62831853, 0.62831853, 0.62831853)

0.6286583105081051

In [16]:
np.linspace(0, 2 * np.pi, 10 + 1)[:-1]

array([0.        , 0.62831853, 1.25663706, 1.88495559, 2.51327412,
       3.14159265, 3.76991118, 4.39822972, 5.02654825, 5.65486678])

In [21]:
energy_scape[1,1,1,1]

0.6286583074948673

In [8]:
def find_landscape_minimas(landscape, filter = True):
    less_right = landscape <= np.roll(landscape, 1 , 0)
    less_left = landscape <= np.roll(landscape, -1 , 0)
    less_gamma = np.logical_and(less_right, less_left)

    less_bottom = landscape <= np.roll(landscape, 1 , 1)
    less_top = landscape <= np.roll(landscape, -1 , 1)
    less_beta = np.logical_and(less_bottom, less_top)

    minimas = np.logical_and(less_gamma, less_beta)

    res = landscape.shape[0]
    s = np.linspace(0, 2 * np.pi, res + 1)[:-1]

    x, y = np.meshgrid([s], [s])
    min_x = x[minimas]
    min_y = y[minimas]
    min_energy = landscape[minimas]

    if filter:
        filtered = min_energy <= np.mean(min_energy)
        min_x = min_x[filtered]
        min_y = min_y[filtered]
        min_energy = min_energy[filtered]

    return np.vstack([min_x, min_y, min_energy])

In [9]:
graph_energy(energy_scape, points = find_landscape_minimas(energy_scape))

In [10]:
def landscape_minimas(landscape, filter = True):

    dims = len(landscape.shape)

    minimas_list = []

    for dim in range(dims):
        less_right = landscape <= np.roll(landscape, 1 , dim)
        less_left = landscape <= np.roll(landscape, -1 , dim)
        minimas_list.append(np.logical_and(less_right, less_left))

    minimas = minimas_list[0].copy()
    for minima in minimas_list[1:]:
        minimas = np.logical_and(minimas, minima)

    res = landscape.shape[0]
    s = np.linspace(0, 2 * np.pi, res + 1)[:-1]

    coords = np.meshgrid(*[s] *  dims)

    points = []

    for coord in coords:
        points.append(coord[minimas])

    min_energy = landscape[minimas]

    if filter:
        filtered = min_energy <= np.mean(min_energy)
        for i in range(dims):
            points[i] = points[i][filtered]
        min_energy = min_energy[filtered]


    return np.vstack(points), min_energy

In [11]:
points, min_energies = landscape_minimas(energy_scape, filter=True)

In [12]:
min_energies < 0

array([False, False, False,  True, False,  True, False,  True, False,
        True, False, False, False, False, False,  True, False,  True,
       False,  True, False,  True, False, False])

In [13]:
points.T[min_energies < 0]

array([[2.26194671, 0.87964594],
       [5.40353936, 0.87964594],
       [1.63362818, 2.38761042],
       [4.77522083, 2.38761042],
       [2.26194671, 4.0212386 ],
       [5.40353936, 4.0212386 ],
       [1.63362818, 5.52920307],
       [4.77522083, 5.52920307]])

In [142]:
dx = 1e-11

def optimize(params):
    params = params.copy()
    gradient = np.ones(params.shape)

    count = 0

    while max(abs(gradient)) >= 1e-2:
        for i in range(len(params)):
            plus_params = params.copy()
            minus_params = params.copy()

            plus_params[i] += dx
            minus_params[i] -= dx
            
            gradient[i] = (numqaoa.energy(*plus_params) - numqaoa.energy(*minus_params)) / (2 * dx)

        count += 1
        params -= gradient * 1e-3
        
        print(count)
        print(numqaoa.energy(*params))

    return params


new_points = []
for point in points.T[min_energies < 0]:
    new_points.append(optimize(point))

1
-3.0257646112604517
2
-3.0377504616627538
3
-3.0386467986225703
4
-3.0387204999710336
5
-3.0387327063401752
6
-3.0387399609940897
7
-3.038746503969706
8
-3.038752680932588
9
-3.03875856771636
10
-3.0387641305907187
11
-3.0387693832536407
12
-3.0387743965971143
13
-3.0387791137647633
14
-3.0387836295396946
15
-3.0387878565747926
16
-3.0387918751915515
17
-3.0387956562700333
18
-3.038799227629695
19
-3.0388026439227547
20
-3.0388058863074345
21
-3.0388089676067502
22
-3.0388118845091543
23
-3.038814657488662
24
-3.038817278390778
25
-3.038819766710467
26
-3.0388220927898733
27
-3.038824313246486
28
-3.038826436984257
29
-3.0388284409038837
30
-3.0388303212117487
31
-3.0388320977180547
32
-3.0388337807428654
33
-3.03883539169358
34
-3.0388369352734883
35
-3.0388383688084986
36
-3.038839735848047
37
-3.038841025283197
38
-3.038842241842173
39
-3.0388433858428314
40
-3.0388444992510313
41
-3.0388455410641413
42
-3.0388465293592755
43
-3.038847474901538
44
-3.0388483688308425
45
-3.0388492

In [145]:
best_points4

array([[1.60249619, 2.3510723 ],
       [4.74408907, 2.35107042],
       [1.60246488, 5.49384802],
       [4.7440572 , 5.49384969]])

In [147]:
best_points6

[array([1.58114956, 2.35371059]),
 array([4.72274265, 2.35372001]),
 array([1.58115049, 5.49525354]),
 array([4.72274363, 5.49523085])]

In [128]:
res = 10
p = 2

s = np.linspace(0, 2 * np.pi, res + 1)[:-1]

dims = np.meshgrid(*[s] * (2 * p))
coords = np.vstack([dim.flatten() for dim in dims]).T

coords.shape

(10000, 4)

In [138]:
np.reshape(coords[:,0], [res] * 4).shape

(10, 10, 10, 10)

In [79]:
scipy.optimize.minimize(
                        fun = lambda x: numqaoa.state_ssd(*x), 
                        x0 = [2.26194671, 0.9424778, 0, 0, 0, 0],
                        method = "L-BFGS-B" ,
                        bounds = [[0, 2 * np.pi] for _ in range(3 * 2)],
                        options = {"gtol" : 1e-2})

  message: CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL
  success: True
   status: 0
      fun: 0.9013844330355459
        x: [ 2.262e+00  9.425e-01  0.000e+00  0.000e+00  0.000e+00
             0.000e+00]
      nit: 1
      jac: [-2.567e-04 -3.481e-03  3.128e+00 -3.481e-03  3.128e+00
            -3.481e-03]
     nfev: 21
     njev: 3
 hess_inv: <6x6 LbfgsInvHessProduct with dtype=float64>

In [21]:
graph_energy(energy_scape,  points = find_landscape_minimas(energy_scape))

In [73]:
class ParametersNode:
    def __init__(self, 
                 parameters,
                 energy):
        
        self.parameters = parameters
        self.energy = energy

        self.id = tuple(np.round(parameters, 4))

        self.p = len(parameters) // 2

        self.gammas = parameters[::2]
        self.betas = parameters[1::2]

        self.layer = None

        self.prev = {}
        self.next = {}

    def optimize_next(self, 
                      energy_fn, 
                      initial_parameters):
        
        next_p = len(initial_parameters) // 2
        
        next_min = scipy.optimize.minimize(
                                    fun = lambda x: energy_fn(*x), 
                                    x0 = initial_parameters,
                                    bounds = [[0, 2 * np.pi] for _ in range(next_p * 2)],
                                    tol = 1e-6)
        
        next_parameters = next_min.x
        next_energy = next_min.fun
        next_node = ParametersNode(next_parameters,
                                   next_energy)
        
        return next_node

class NodeLayer:
    def __init__(self):
        self.nodes = {}
        self.num_nodes = 0

        self.prev = None
        self.next = None
        
    def optimize_next(self, energy_fn, parameter_fn):
        for node in self.nodes.values():
            initial_parameters = parameter_fn(node.parameters)
            
            new_node = node.optimize_next(energy_fn, initial_parameters)

            if self.next is None:
                self.next = NodeLayer()
                self.next.prev = self

            # if parameter already exists, throw away current node and reference to originally created node
            if new_node.id in self.next.nodes.keys():
                new_node = self.next.nodes[new_node.id]
            else:
                self.next.add_node(new_node)

            node.next[new_node.id] = new_node
            new_node.prev[node.id] = node

    def add_node(self, new_node: ParametersNode):
        self.nodes[new_node.id] = new_node
        self.num_nodes = len(self.nodes)

In [34]:
def closeness_to_ground(val):
    return 1 - (ham.eigen_values()[-1] - val) / (ham.eigen_values()[-1] - ham.eigen_values()[0])

def find_minimas(hamiltonian, 
                 verbose = True):
    
    start = time.time()
    layers = []

    ### Find p = 1 minimas ###

    symqaoa = QAOA_Sympy(ham, p=1)

    if verbose: print(f"Making p = 1 Expectation {time.time() - start}")
    symqaoa.make()

    if verbose: print(f"Making p = 1 Landscape {time.time() - start}")
    sym_energies = symqaoa.energy_landscape(res=128)

    energy_minimas = find_landscape_minimas(sym_energies)

    energy_min_x, energy_min_y, min_energies = energy_minimas

    minimas = []

    for x, y in zip(energy_min_x, energy_min_y):
        minimas.append(scipy.optimize.minimize(fun = lambda x: symqaoa.energy(*x), 
                                x0 = [x, y],
                                bounds = [[0, 2 * np.pi] for _ in range(2)]))
        
    avg_energy = 0
    for minima in minimas:
        avg_energy += minima.fun 
    avg_energy /= len(minimas)

    minimas = [minima for minima in minimas if minima.fun < avg_energy]

    layers.append(NodeLayer())


    if verbose:print("\np = 1 energies:")
    for minima in minimas:
        if verbose: print(f"{minima.fun}")
        node = ParametersNode(minima.x, minima.fun)
        layers[0].add_node(node)
    if verbose:print("")

    ### Find p = 2 minimas ###

    if verbose: print(f"Making p = 2 Expectation {time.time() - start}")
    symqaoa.add_round()
    symqaoa.make_expectation()
    
    def perturb_params(perturbation):
        def parameter_fn(parameters):
            new_parameters = np.hstack([parameters, parameters])
            return new_parameters + perturbation
        return parameter_fn
    
    if verbose: print(f"Optimizing p = 2 {time.time() - start}")
    magnitudes = [0.2, 1]
    for magnitude in magnitudes:
        perturbations = [0, magnitude, -magnitude]
        for pgamma in perturbations:
            for pbeta in perturbations:
                parameter_fn = perturb_params(np.array([0, 0, pgamma, pbeta]))
                layers[0].optimize_next(symqaoa.energy, parameter_fn)

    layers.append(NodeLayer())

    prior_min_energy = min([node.energy for node in layers[0].nodes.values()])

    if verbose: print("\np = 2 energies:")
    for node in layers[0].next.nodes.values():
        if node.energy <= prior_min_energy:
            if verbose: print(node.energy)
            layers[1].add_node(node)
    if verbose: print("")

    print(f"Total Time: {time.time() - start}")

    ### Find p >= 3 minimas ###

    def interpolate_params(parameters):
        p = len(parameters) // 2
        gammas, betas = parameters[::2], parameters[1::2]

        x_fit = np.linspace(0, 1, p)
        x_poly = np.linspace(0, 1, p + 1)

        gamma_interp = np.interp(x_poly, x_fit, gammas)
        beta_interp = np.interp(x_poly, x_fit, betas)
        new_parameters = np.array([val for pair in zip(gamma_interp, beta_interp) for val in pair])
        return new_parameters

    def fit_params(parameters):
        p = len(parameters) // 2
        gammas, betas = parameters[::2], parameters[1::2]

        x_fit = np.linspace(0, 1, p)
        x_poly = np.linspace(0, 1, p + 1)

        gamma_fit = np.poly1d(np.polyfit(x_fit, gammas, 2))
        beta_fit = np.poly1d(np.polyfit(x_fit, betas, 2))

        new_parameters = np.array([val for pair in zip(gamma_fit(x_poly), beta_fit(x_poly)) for val in pair])
        return new_parameters
    
    # numqaoa = QAOA_Numpy(ham)
    
    # for i in range(1, 4):
    #     if verbose: print(f"Optimizing p = {i + 1} {time.time() - start}")

    #     layers[i].optimize_next(numqaoa.energy, fit_params)
    #     layers[i].optimize_next(numqaoa.energy, interpolate_params)

    #     layers.append(NodeLayer())

    #     prior_min_energy = min([node.energy for node in layers[i].nodes.values()])

    #     if verbose: print(f"\np = {i + 1} energies:")
    #     for node in layers[i].next.nodes.values():
    #         if node.energy <= prior_min_energy:
    #             if verbose: print(node.energy)
    #             layers[i + 1].add_node(node)
    #     if verbose: print("")

    return layers

In [35]:
layers = find_minimas(ham)

Making p = 1 Expectation 0.009225845336914062
Making p = 1 Landscape 0.0951688289642334

p = 1 energies:
-0.9675454616572096
-0.9675454616572094
-1.0864171255830373
-1.086417125583036
-0.9675454616572096
-0.9675454616572096
-1.0864171255830335
-1.0864171255830344

Making p = 2 Expectation 0.14508891105651855
Optimizing p = 2 0.3895599842071533

p = 2 energies:
-1.099586409039816
-1.0995864090410512
-1.099586398878394
-1.0995863988778523
-1.0877379459635133
-1.0877379459782317
-1.0995864049986372
-1.0995863761987537
-1.0995864055509108
-1.0995863912478827
-1.0995863957051832
-1.099586405488631
-1.0995864058188
-1.0995864087052238
-1.099586401022923
-1.0995863275530704
-1.0995861507125917
-1.0995863754415727
-1.0995863950206004
-1.0995863074622083
-1.0877380011791777
-1.0995864085641183
-1.0877379427495466
-1.0995864094166474
-1.0995863584788979
-1.0995863584710601
-1.0995863584983765
-1.099586358507642
-1.0877379606157693
-1.0877370802123478
-1.099586407453272
-1.0877380050491807
-1.087

In [19]:
(1 - closeness_to_ground(-1.0864171255830373)) * 100

99.51209927196535

In [26]:
layers[0].next.nodes

{(2.1909, 0.7594, 1.7435, 0.2462): <__main__.ParametersNode at 0x2a3cee830>,
 (5.3325, 0.7594, 4.8851, 0.2462): <__main__.ParametersNode at 0x2a3cec310>,
 (1.5753, 2.475, 2.2291, 3.1416): <__main__.ParametersNode at 0x2a3cec040>,
 (4.7169, 2.475, 5.3707, 3.1416): <__main__.ParametersNode at 0x2a3cecd90>,
 (2.1909, 3.901, 1.7435, 3.3879): <__main__.ParametersNode at 0x2a3cec7c0>,
 (5.3325, 3.901, 4.8851, 3.3879): <__main__.ParametersNode at 0x2a3ced0f0>,
 (1.5753, 5.6164, 2.2293, 6.2832): <__main__.ParametersNode at 0x2a3cedb40>,
 (4.7169, 5.6164, 5.3708, 6.2832): <__main__.ParametersNode at 0x2a3ced150>,
 (0.068, 0.0, 0.8566, 0.7853): <__main__.ParametersNode at 0x2a3ced5a0>,
 (3.4844, 0.0, 4.6713, 0.0): <__main__.ParametersNode at 0x2a3ced030>,
 (1.5753, 2.4749, 2.2289, 3.1415): <__main__.ParametersNode at 0x2a3ced930>,
 (4.7169, 2.4749, 5.3705, 3.1415): <__main__.ParametersNode at 0x2a3ced960>,
 (0.0, 2.5664, 0.0, 1.3606): <__main__.ParametersNode at 0x2a3ceda50>,
 (2.809, 1.5708, 3.

In [55]:
def interpolate_params(parameters):
        p = len(parameters) // 2
        gammas, betas = parameters[::2], parameters[1::2]

        x_fit = np.linspace(0, 1, p)
        x_poly = np.linspace(0, 1, p + 1)

        gamma_interp = np.interp(x_poly, x_fit, gammas)
        beta_interp = np.interp(x_poly, x_fit, betas)
        new_parameters = np.array([val for pair in zip(gamma_interp, beta_interp) for val in pair])
        return new_parameters

def fit_params(parameters):
    p = len(parameters) // 2
    gammas, betas = parameters[::2], parameters[1::2]

    x_fit = np.linspace(0, 1, p)
    x_poly = np.linspace(0, 1, p + 1)

    gamma_fit = np.poly1d(np.polyfit(x_fit, gammas, 2))
    beta_fit = np.poly1d(np.polyfit(x_fit, betas, 2))

    new_parameters = np.array([val for pair in zip(gamma_fit(x_poly), beta_fit(x_poly)) for val in pair])
    return new_parameters

numqaoa = QAOA_Numpy(ham)

verbose = True

for i in range(5, 20):
    if verbose: print(f"Fit p = {i + 2}")
    layers[i].optimize_next(numqaoa.energy, fit_params)
    if verbose: print(f"Interpolate p = {i + 2}")
    layers[i].optimize_next(numqaoa.energy, interpolate_params)

    layers.append(NodeLayer())

    prior_min_energy = min([node.energy for node in layers[i].nodes.values()])

    if verbose: print(f"\np = {i + 1} energies:")
    for node in layers[i].next.nodes.values():
        if node.energy < prior_min_energy:
            if verbose: print(node.energy)
            layers[i + 1].add_node(node)
    if verbose: print("")

    print(min([node.energy for node in layers[i].next.nodes.values()]))

    with open('data', 'wb') as f:
        pickle.dump(layers,f)

Fit p = 7
Interpolate p = 7

p = 6 energies:
-2.1558553599335792
-2.155887944159434
-2.1558883688381383
-2.1558906597753302
-2.1558827654058907
-2.1558695793653473
-2.1558549261576854
-2.1559030225913727
-2.155851671448172
-2.1559501814455233

-2.1559501814455233
Fit p = 8
Interpolate p = 8

p = 7 energies:
-2.1559656974499584

-2.1559656974499584
Fit p = 9
Interpolate p = 9

p = 8 energies:

-2.155857707711967
Fit p = 10
Interpolate p = 10


ValueError: min() arg is an empty sequence

In [104]:
converged_parameters = []
converged_energies = []

def find_converged_parameters(node):
    converged_energies.append(node.energy)
    converged_parameters.append(node.parameters)
    if node.p > 1:
        for node in node.prev.values():
            find_converged_parameters(node)
            

for i, node in enumerate(layers[7].next.nodes.values()):
        if i == 1:
            find_converged_parameters(node)

In [105]:
converged_energies

[-2.155857707711967,
 -2.1559656974499584,
 -2.1558553599335792,
 -2.155522553277559,
 -2.154824109487455,
 -2.1309447572121893,
 -2.120272801847749,
 -2.109174241721617,
 -2.07964928701366]

In [106]:
def closeness_to_ground(val):
    return 1 - (ham.eigen_values()[-1] - val) / (ham.eigen_values()[-1] - ham.eigen_values()[0])

closeness_to_ground(np.array(converged_energies)) * 100

array([1.87232156e-03, 1.63165820e-04, 1.90947989e-03, 7.17681748e-03,
       1.82311013e-02, 3.96170086e-01, 5.65075337e-01, 7.40732470e-01,
       1.20802452e+00])

In [33]:
ham.eigen_values()[0]

-2.1559760067703464

In [131]:
import plotly.graph_objects as go

def plot_parameters(converged_parameters):
    fig = go.Figure()

    num_lines = len(converged_parameters)

    for i, parameters in enumerate(converged_parameters[::-1]):
        p = len(parameters) // 2

        gammas = parameters[::2]
        betas = parameters[1::2]

        print(p)
        
        fig.add_trace(go.Scatter(x=np.linspace(0, 1, p), y = gammas,
                                 opacity= ((i + 1) / num_lines) ** 5,
                                 mode="lines",
                                 line=dict(color='red')))
        fig.add_trace(go.Scatter(x=np.linspace(0, 1, p), y = betas,
                                 opacity= ((i + 1) / num_lines) ** 5,
                                 mode="lines",
                                 line=dict(color='blue')))
    fig.update_layout(showlegend=False, 
                    width=1200,
                    height=700,
                    title = "Parameter Curves",
                    xaxis_title = "Step (i-1)/(p-1)",
                    yaxis_title = "Gamma(red), Beta(blue)")
            
    fig.show()
    
plot_parameters(converged_parameters)

1
2
3
4
5
6
7
8
9


In [138]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=list(range(1, 1 +len(converged_energies))), y = (1 - closeness_to_ground(np.array(converged_energies[1::][::-1]))) * 100,
                                 line=dict(color='violet')))

fig.update_layout(showlegend=False,
                  width=1200,
                    height=700,
                  title = "Performance Scaling of p",
                  xaxis_title = "p - number of rounds",
                  yaxis_title = "Closeness to Ground (%)")

fig.show()

In [139]:
(1 - closeness_to_ground(np.array(converged_energies[1::][::-1]))) * 100

array([98.79197548, 99.25926753, 99.43492466, 99.60382991, 99.9817689 ,
       99.99282318, 99.99809052, 99.99983683])

In [137]:
closeness_to_ground(np.array(converged_energies[1::][::-1]))[-1]

1.6316581957775966e-06

In [68]:
def closeness_to_ground(val):
    return (ham.eigen_values()[-1] - val) / (ham.eigen_values()[-1] - ham.eigen_values()[0])

closeness_to_ground(ham.eigen_values()[0] + 1e-4) * 100

99.99629516126933

In [21]:
with open('data', 'wb') as f:
    pickle.dump(layers,f)

with open('data', 'rb') as f:
    y = pickle.load(f)

In [144]:
numqaoa.energy(*converged_parameters[0])

-2.155857707711967

In [158]:
numqaoa.energy(*(np.random.rand(18) * 2 * np.pi))

0.5813156838282922

In [159]:
scipy.optimize.minimize(
                        fun = lambda x: numqaoa.energy(*x), 
                        x0 = np.random.rand(18) * 2 * np.pi,
                        bounds = [[0, 2 * np.pi] for _ in range(9 * 2)],
                        tol = 1e-6)

  message: CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH
  success: True
   status: 0
      fun: -2.1389073211120793
        x: [ 0.000e+00  5.622e+00 ...  6.566e-01  2.418e+00]
      nit: 111
      jac: [ 4.540e-01  1.935e-03 ... -9.271e-03 -5.471e-03]
     nfev: 2451
     njev: 129
 hess_inv: <18x18 LbfgsInvHessProduct with dtype=float64>

In [28]:
import multiprocessing
def printVal(value):
    return ("hello "*(value))
pool_obj = multiprocessing.Pool()
ans = pool_obj.map(printVal,range(0,4))
print(ans)
pool_obj.close()

Process SpawnPoolWorker-17:
Process SpawnPoolWorker-18:
Process SpawnPoolWorker-19:
Process SpawnPoolWorker-20:
Traceback (most recent call last):
  File "/Users/riley/anaconda3/envs/qc/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/Users/riley/anaconda3/envs/qc/lib/python3.10/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/riley/anaconda3/envs/qc/lib/python3.10/multiprocessing/pool.py", line 114, in worker
    task = get()
  File "/Users/riley/anaconda3/envs/qc/lib/python3.10/multiprocessing/queues.py", line 367, in get
    return _ForkingPickler.loads(res)
AttributeError: Can't get attribute 'printVal' on <module '__main__' (built-in)>
Traceback (most recent call last):
  File "/Users/riley/anaconda3/envs/qc/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/Users/riley/anaconda3/envs/qc/lib/python3.10/multiprocessing/process.py", line 108, 

KeyboardInterrupt: 

In [36]:
np.pi * np.array([1e-3, 1e-2, 2e-2, 1e-1, 2e-1, 5e-1, 1])

array([0.00314159, 0.03141593, 0.06283185, 0.31415927, 0.62831853,
       1.57079633, 3.14159265])

In [37]:
4.74407226 - 1.60247959

3.1415926700000005

In [44]:
np.linspace(0, np.pi, 20)[1:-1]

array([0.16534698, 0.33069396, 0.49604095, 0.66138793, 0.82673491,
       0.99208189, 1.15742887, 1.32277585, 1.48812284, 1.65346982,
       1.8188168 , 1.98416378, 2.14951076, 2.31485774, 2.48020473,
       2.64555171, 2.81089869, 2.97624567])

In [55]:
np.log10(0)

-inf

In [57]:
np.array([3, 2, 5]) % 2

array([1, 0, 1])

In [59]:
ham.eigen_values()[0] + 1e-4

-1.099486409417042